In [1]:
%matplotlib nbagg
#%load_ext line_profiler
#%load_ext memory_profiler

import jungfrau_utils as ju
import numpy as np
import json
import os, math
import time
from matplotlib import pyplot as plt
from matplotlib import cm
from scipy.optimize import curve_fit
from IPython.display import clear_output, display
import matplotlib.colors as colors

import h5py
from ipyfilechooser import FileChooser

from alvra_tools import *
from alvra_tools.channels import *
from alvra_tools.utils import errfunc_sigma, errfunc_fwhm, threshold
import glob

In [14]:
datafiles = []
for file in glob.glob('/sf/alvra/data/p18938/raw/scan_info/Febpy_XES_timescan_NOPA*'):
    datafiles.append(file)
datafiles = sorted(datafiles, key = os.path.getmtime)
patch = datafiles[24:30]
patch

['/sf/alvra/data/p18938/raw/scan_info/Febpy_XES_timescan_NOPA_loop_23.json',
 '/sf/alvra/data/p18938/raw/scan_info/Febpy_XES_timescan_NOPA_loop_24.json',
 '/sf/alvra/data/p18938/raw/scan_info/Febpy_XES_timescan_NOPA_loop_25.json',
 '/sf/alvra/data/p18938/raw/scan_info/Febpy_XES_timescan_NOPA_loop_26.json',
 '/sf/alvra/data/p18938/raw/scan_info/Febpy_XES_timescan_NOPA_loop_27.json',
 '/sf/alvra/data/p18938/raw/scan_info/Febpy_XES_timescan_NOPA_loop_28.json']

In [15]:
roi1 = (7740, 7900, 220, 300) #signal 1
roi2 = (7740, 7900, 400, 480) #bkg 1

roi3 = (5080, 5220, 180, 280) #signal 2
roi4 = (5080, 5220, 300, 400) #bkg 2

threshold_low = 5
#threshold_high = 10

######################################################################################
outdir = '/sf/alvra/data/p18938/res/Claudio/'

XAS_total_roi1 = 0
XAS_total_roi3 = 0

clock2 = Clock()

for json_file in patch:
    XAS_roi1 = []
    XAS_roi1_on = []
    #Error_XAS_roi1 = []
    XAS_roi3 = []
    XAS_roi3_on = []
    #Error_XAS_roi3 = []
        
    RIXS_transient_roi1 = []
    RIXS_transient_roi3 = []
    Error_transient_roi1 = []
    Error_transient_roi3 = []
    RIXS_unpumped_roi1 = []
    RIXS_unpumped_roi3 = []
    Error_unpumped_roi1 = []
    Error_unpumped_roi3 = []
    RIXS_pumped_roi1 = []
    RIXS_pumped_roi3 = []
    Error_pumped_roi1 = []
    Error_pumped_roi3 = []
    XAS_transient_roi1 =[]
    XES_transient_roi1 =[]
    XAS_transient_roi3 =[]
    XES_transient_roi3 =[]
        
    XES_abs_roi1 = []
    XES_abs_roi3 = []

    scan_data = load_scan_data(json_file)

    Energy_eV = []
    Delay_mm = load_scan_readback(json_file)
    
    #XES_1D = []
    
    clock1 = Clock()
    
    num_scan_files = len(scan_data)
    print('json file is {}'.format(json_file))
    print("\nProcessing", num_scan_files, "total files")
    
    for i, (pv, bs, fn) in enumerate(scan_data):
    #for i in range(0,4):
        print("\n File {} out of {}: Processing {}".format(i+1, num_scan_files, fn.split('/')[-2:]))
        
        spectrum_ON_roi1 = 0
        spectrum_ON_roi2 = 0
        spectrum_ON_roi3 = 0
        spectrum_ON_roi4 = 0
        spectrum_OFF_roi1 = 0
        spectrum_OFF_roi2 = 0
        spectrum_OFF_roi3 = 0
        spectrum_OFF_roi4 = 0
        
        nshots = None  # put this to None for ALL images
        #nshots = None
        
        
        images_pump_roi1, images_pump_roi2, images_pump_roi3, images_pump_roi4, pids_ON, \
        images_unpump_roi1, images_unpump_roi2, images_unpump_roi3, images_unpump_roi4, pids_OFF = \
        load_crop_JF_batches_on_off_4rois(fn, roi1, roi2, roi3, roi4, 50, 25, nshots=nshots)
        
    #   images_ON_roi1, images_ON_roi2, pids_ON, images_OFF_roi1, images_OFF_roi2, pids_OFF = \
    #   load_crop_JF_data_on_off(fn, roi1, roi2, 50, 25, pedestal_file=pedestal_file, nshots=nshots)
        
    #    energy, pids = load_single_channel_pulseID(bs, channel_energy, 50)
    #    energy = [x for x in energy if (x > 0)]
    #    Energy_eV.append(np.mean(energy))
        
    #    delay, pids = load_single_channel_pulseID(bs, channel_delay_harm, 50)
    #    delay = [x for x in delay if (x > 0)]
    #    Delay_mm.append(np.mean(delay))
    
        images_pump_roi1_thr = images_pump_roi1.copy()
        images_pump_roi1_thr[images_pump_roi1 < threshold_low] = 0
        #images_pump_roi1_thr[images_pump_roi1 > threshold_high] = 0
        images_unpump_roi1_thr = images_unpump_roi1.copy()
        images_unpump_roi1_thr[images_unpump_roi1 < threshold_low] = 0
        #images_unpump_roi1_thr[images_unpump_roi1 > threshold_high] = 0
        
        images_pump_roi2_thr = images_pump_roi2.copy()
        images_pump_roi2_thr[images_pump_roi2 < threshold_low] = 0
        #images_pump_roi2_thr[images_pump_roi2 > threshold_high] = 0
        images_unpump_roi2_thr = images_unpump_roi2.copy()
        images_unpump_roi2_thr[images_unpump_roi2 < threshold_low] = 0
        #images_unpump_roi2_thr[images_unpump_roi2 > threshold_high] = 0
        
        images_pump_roi3_thr = images_pump_roi3.copy()
        images_pump_roi3_thr[images_pump_roi3 < threshold_low] = 0
        #images_pump_roi3_thr[images_pump_roi3 > threshold_high] = 0
        images_unpump_roi3_thr = images_unpump_roi3.copy()
        images_unpump_roi3_thr[images_unpump_roi3 < threshold_low] = 0
        #images_unpump_roi3_thr[images_unpump_roi3 > threshold_high] = 0
        
        images_pump_roi4_thr = images_pump_roi4.copy()
        images_pump_roi4_thr[images_pump_roi4 < threshold_low] = 0
        #images_pump_roi4_thr[images_pump_roi4 > threshold_high] = 0
        images_unpump_roi4_thr = images_unpump_roi4.copy()
        images_unpump_roi4_thr[images_unpump_roi4 < threshold_low] = 0
        #images_unpump_roi4_thr[images_unpump_roi4 > threshold_high] = 0
        
        spectrum_ON_roi1 = images_pump_roi1_thr.sum(axis = 1).sum(axis = 0)
        spectrum_OFF_roi1 = images_unpump_roi1_thr.sum(axis = 1).sum(axis = 0)
        spectrum_ON_roi2 = images_pump_roi2_thr.sum(axis = 1).sum(axis = 0)
        spectrum_OFF_roi2 = images_unpump_roi2_thr.sum(axis = 1).sum(axis = 0)
        spectrum_ON_roi3 = images_pump_roi3_thr.sum(axis = 1).sum(axis = 0)
        spectrum_OFF_roi3 = images_unpump_roi3_thr.sum(axis = 1).sum(axis = 0)
        spectrum_ON_roi4 = images_pump_roi4_thr.sum(axis = 1).sum(axis = 0)
        spectrum_OFF_roi4 = images_unpump_roi4_thr.sum(axis = 1).sum(axis = 0)
    
    ######################### ROI 1 & 2 ##############################################
        
        XAS_roi1.append(np.abs(spectrum_OFF_roi1.sum() - spectrum_OFF_roi2.sum()))
        XAS_roi1_on.append(np.abs(spectrum_ON_roi1.sum() - spectrum_ON_roi2.sum()))
        RIXS_unpumped_roi1.append(spectrum_OFF_roi1 - spectrum_OFF_roi2)
     #   Error_unpumped_roi1.append(np.sqrt(spectrum_OFF_roi1))
        RIXS_pumped_roi1.append(spectrum_ON_roi1 - spectrum_ON_roi2)
     #   Error_pumped_roi1.append(np.sqrt(spectrum_ON_roi1))
    
        XES_transient_roi1 = (spectrum_ON_roi1 - spectrum_ON_roi2) - (spectrum_OFF_roi1 - spectrum_OFF_roi2)
        
        RIXS_transient_roi1.append(XES_transient_roi1)
     #   Error_transient_roi1.append(np.sqrt((np.sqrt(spectrum_ON_roi1))**2 + (np.sqrt(spectrum_OFF_roi1))**2))
      
        XES_abs_roi1 = np.abs(XES_transient_roi1)
        
     #   XES_1D.append(np.sum(XES_transient_roi1, axis = 1))
        
        XAS_transient_roi1.append(np.sum(XES_abs_roi1[50:80]))
        
    ######################### ROI 3 & 4 ##############################################
        
        XAS_roi3.append(np.abs(spectrum_OFF_roi3.sum() - spectrum_OFF_roi4.sum()))
        XAS_roi3_on.append(np.abs(spectrum_ON_roi3.sum() - spectrum_ON_roi4.sum()))
        RIXS_unpumped_roi3.append(spectrum_OFF_roi3 - spectrum_OFF_roi4)
     #   Error_unpumped_roi3.append(np.sqrt(spectrum_OFF_roi3))
        RIXS_pumped_roi3.append(spectrum_ON_roi3 - spectrum_ON_roi4)
     #   Error_pumped_roi3.append(np.sqrt(spectrum_ON_roi3))
    
        XES_transient_roi3 = (spectrum_ON_roi3 - spectrum_ON_roi4) - (spectrum_OFF_roi3 - spectrum_OFF_roi4)
        
        RIXS_transient_roi3.append(XES_transient_roi3)
     #   Error_transient_roi3.append(np.sqrt((np.sqrt(spectrum_ON_roi1))**2 + (np.sqrt(spectrum_OFF_roi1))**2))
        
        XES_abs_roi3 = np.abs(XES_transient_roi3)
     #   XES_1D.append(np.sum(XES_transient_roi3, axis = 1))
        
        XAS_transient_roi3.append(np.sum(XES_abs_roi3))
        
        clear_output(wait=True)
        
        print ("It took {} to process file {}".format(clock1.tick(), fn.split('/')[-1]))
        
    filename_h5 = outdir + json_file.split('/')[-1].split('.')[0] + "_export.h5" 
    with h5py.File(filename_h5, "w") as file2write:
        file2write.create_dataset("RIXS_pumped_roi1", data = RIXS_pumped_roi1)
        file2write.create_dataset("RIXS_unpumped_roi1", data = RIXS_unpumped_roi1)
        file2write.create_dataset("RIXS_transient_roi1", data = RIXS_transient_roi1)
        file2write.create_dataset("RIXS_pumped_roi3", data = RIXS_pumped_roi3)
        file2write.create_dataset("RIXS_unpumped_roi3", data = RIXS_unpumped_roi3)
        file2write.create_dataset("RIXS_transient_roi3", data = RIXS_transient_roi3)        
    print(filename_h5)
    
    XAS_total_roi1 += np.array(XAS_transient_roi1)
    XAS_total_roi3 += np.array(XAS_transient_roi3)
    
    
    print ("\nJob done! It took", clock1.tock(), "seconds to process", num_scan_files, "file(s)")

Energy_eV = np.array(Energy_eV)
Delay_mm = np.array(Delay_mm)
XAS_roi1 = np.array(XAS_roi1)
XAS_roi1_on = np.array(XAS_roi1_on)
RIXS_unpumped_roi1 = np.array(RIXS_unpumped_roi1)
Error_unpumped_roi1 = np.array(Error_unpumped_roi1)
RIXS_pumped_roi1 = np.array(RIXS_pumped_roi1)
Error_pumped_roi1 = np.array(Error_pumped_roi1)
RIXS_transient_roi1 = np.array(RIXS_transient_roi1)
Error_transient_roi1 = np.array(Error_transient_roi1)
XAS_transient_roi1=np.array(XAS_transient_roi1)
XES_transient_roi1=np.array(XES_transient_roi1)
#XES_1D=np.array(XES_1D)
XAS_roi3 = np.array(XAS_roi3)
XAS_roi3_on = np.array(XAS_roi3_on)
RIXS_unpumped_roi3 = np.array(RIXS_unpumped_roi3)
Error_unpumped_roi3 = np.array(Error_unpumped_roi3)
RIXS_pumped_roi3 = np.array(RIXS_pumped_roi3)
Error_pumped_roi3 = np.array(Error_pumped_roi3)
RIXS_transient_roi3 = np.array(RIXS_transient_roi3)
Error_transient_roi3 = np.array(Error_transient_roi3)
XAS_transient_roi3=np.array(XAS_transient_roi3)
XES_transient_roi3=np.array(XES_transient_roi3)
#XES_1D=np.array(XES_1D)

XAS_total_roi1 = np.asarray(XAS_total_roi1)
XAS_total_roi3 = np.asarray(XAS_total_roi3)

print ("\nJob done! It took", clock2.tick(), "seconds to process {} json files".format(len(patch)))


It took 11.52 to process file run_012899.JF02T09V02.h5
/sf/alvra/data/p18938/res/Claudio/Febpy_XES_timescan_NOPA_loop_28_export.h5

Job done! It took 366.05 seconds to process 31 file(s)

Job done! It took 2218.03 seconds to process 6 json files
